In [8]:
#############################################################
# 1. Libraries

import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
import glob
from tqdm.notebook import tqdm
import pickle
from collections import defaultdict
import time

import tensorflow as tf
tf.keras.backend.clear_session()

physical_devices = tf.config.list_physical_devices('GPU')

try:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)
except:
    print('Invalid device or cannot modify virtual devices once initialized.')

from tensorflow.keras import models, layers, regularizers, metrics, losses, optimizers, constraints
from tensorflow.keras.utils import Sequence

from sklearn.model_selection import KFold, GroupKFold, StratifiedKFold
from sklearn.metrics import accuracy_score

#############################################################

In [9]:
#############################################################
# 1. Utils

class Experiment(object):
    def __init__(self, **kwargs):
        self.__dict__ = kwargs

    def __repr__(self):
        return str(self.__dict__)
    
    
def euclideanDistance(xp, x, yp, y):
    return tf.reduce_mean(tf.math.sqrt(tf.square(xp - x) + tf.square(yp - y)))

def meanAbsoluteError(y_true, y_pred):
    return tf.reduce_mean(tf.abs(y_true - y_pred))

def meanSquaredError(y_true, y_pred):
    return tf.reduce_mean(tf.square(y_true - y_pred))

    
def customLossFunction(floor, x, y, p_floor, p_x, p_y):
    penalty_ = tf.constant(15.0, dtype=tf.float32)
    
    p_floor = tf.math.round(p_floor)
    
    ed_ = euclideanDistance(p_x, x, p_y, y)
    f_ = tf.abs(floor - p_floor)
    loss_ = ed_ + penalty_ * f_
    return tf.reduce_mean(loss_)


def normalize(x, mean_, std_):
    return (x - mean_) / std_

def denormalize(x, mean_, std_):
    return (x * std_) + mean_


def plotMetrics(history, figsize=(12, 8)):
    # Accuracy
    plt.figure(figsize=figsize)
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'val'], loc='upper left')
    plt.show()
    
    # Loss
    plt.figure(figsize=figsize)
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'val'], loc='upper left')
    plt.show()
    
#############################################################

In [10]:
#################################################
# 1. Experiment & Global Variables & Paths

# Paths 

PATH_DATA = '../../01_Data/'
PATH_MODELS = '../../03_Models/'
PATH_PREDICTIONS = '../../04_Predictions/'

path_data_train = PATH_DATA + 'train/'
path_data_test = PATH_DATA + 'test/'
path_metadata = PATH_DATA + 'metadata/'

path_generated_data = PATH_DATA + 'GeneratedData/v0.2/'
path_generated_data_train = path_generated_data + 'train/'
path_generated_data_test = path_generated_data + 'test/'
path_output_data_images = PATH_DATA + 'GeneratedData/v0.2/images/'

# Global Variables

DICT_FLOOR_MAP = {'1F' :  0, '2F' : 1, '3F' : 2, '4F' : 3, '5F' : 4, 
                     '6F' : 5, '7F' : 6, '8F' : 7, '9F' : 8,
                     'B'  : -1, 'B1' : -1, 'B2' : -2, 'B3' : -3, 
                     'BF' : -1, 'BM' : -1, 
                     'F1' : 0, 'F2' : 1, 'F3' : 2, 'F4' : 3, 'F5' : 4, 
                     'F6' : 5, 'F7' : 6, 'F8' : 7, 'F9' : 8, 'F10': 9,
                     'L1' : 0, 'L2' : 1, 'L3' : 2, 'L4' : 3, 'L5' : 4, 
                     'L6' : 5, 'L7' : 6, 'L8' : 7, 'L9' : 8, 'L10': 9, 
                     'L11': 10,
                     'G'  : 0, 'LG1': 0, 'LG2': 1, 'LM' : 0, 'M'  : 0, 
                     'P1' : 0, 'P2' : 1,}

VERSION = '009'

list_train_paths = glob.glob(path_generated_data_train + '*')
list_test_paths = glob.glob(path_generated_data_test + '*')
list_metadata_paths = glob.glob(path_metadata + '*')

list_train_traces = [trace.split('\\')[-1] for trace in list_train_paths]
list_test_traces = [trace.split('\\')[-1] for trace in list_test_paths]

assert len(list_train_paths)==len(list_train_traces)
assert len(list_test_paths)==len(list_test_traces)

dict_train_traces_paths = {trace : path for trace, path in zip(list_train_traces, list_train_paths)}
dict_test_traces_paths = {trace : path for trace, path in zip(list_test_traces, list_test_paths)}

# Load experiment
with open(f'{path_generated_data}Experiment_{VERSION}.pkl', 'rb') as f:
    experiment = pickle.load(f)
    
print(f'Version: {experiment.version}')


#Embedding dims

EMB_SITE_DIM = len(experiment.dict_all_sites)
EMB_UUID_DIM = len(experiment.dict_unique_uuid)
EMB_BSSID_DIM = len(experiment.dict_unique_bssid)
EMB_SSID_DIM = len(experiment.dict_unique_ssid)

# Floors

dict_training_floor = {}
for trace in tqdm(list_train_traces):
    path = dict_train_traces_paths[trace]
    data = np.load(f'{path}/waypoint_predict.npy', allow_pickle=True).astype(np.float32)
    floor = data[:, 0][0]
    dict_training_floor[trace] = floor
    
    
# Trace + Site + Floor
dict_site_floor_traces = {}
list_site_floor_traces = glob.glob(PATH_DATA  + 'train/' + '/*/*/*.txt')
for file in tqdm(list_site_floor_traces):
    list_file = file.split('\\')
    site = list_file[1]
    floor = list_file[2]
    trace = list_file[-1].replace('.txt', '')
    dict_site_floor_traces[trace] = (site, floor)
    
    
# For each site, floor how many traces there are
dict_train_site_floor_traces = {}
for trace in tqdm(dict_train_traces_paths):
    labels = np.load(f'{dict_train_traces_paths[trace]}/waypoint_predict.npy', allow_pickle=True).astype(np.float32)
    site, floor = dict_site_floor_traces[trace]
    if site not in dict_train_site_floor_traces:
        dict_train_site_floor_traces[site] = {
            floor : [trace]
        }
    elif floor not in dict_train_site_floor_traces[site]:
        dict_train_site_floor_traces[site][floor] = [trace]
    else:
        dict_train_site_floor_traces[site][floor].extend([trace])
        
# build train val trraces        
list_train_train_traces, list_train_val_traces = [], []
for site in tqdm(dict_train_site_floor_traces):
    num_floors_site = len(dict_train_site_floor_traces[site])
    for floor in dict_train_site_floor_traces[site]:
        if num_floors_site==1:
            list_traces = dict_train_site_floor_traces[site][floor]
            list_train_train_traces.extend(list_traces)
        else:
            list_traces = dict_train_site_floor_traces[site][floor]
            np.random.shuffle(list_traces)
            num_train = int(len(list_traces) * 0.95) + 1
            list_train_train_traces.extend(list_traces[:num_train])
            list_train_val_traces.extend(list_traces[num_train:])
            
            
pc_val_traces = len(list_train_val_traces) / (len(list_train_val_traces)+len(list_train_train_traces))
print(pc_val_traces)
print(len(list_train_train_traces), len(list_train_val_traces), 
      len(list_train_train_traces) + len(list_train_val_traces), len(list_train_traces))

# MIN_FLOOR = min(experiment.list_floors_train_filtered)
# MAX_FLOOR = max(experiment.list_floors_train_filtered)
NUM_FLOORS = len(experiment.list_floors_train_filtered)
# DICT_MAP_FLOORS = {f : i for i, f in enumerate(experiment.list_floors_train_filtered)}

#################################################

Version: 009



0.034499672988881624
23620 844 24464 24464


In [11]:
#################################################
# 3. Functions

def getAngles(pos, i, d_model):
    angle_rates = 1 / np.power(10000, (2 * (i//2)) / np.float32(d_model))
    return pos * angle_rates

def positionalEncoding(position, trf_dim):
    angle_rads = getAngles(np.arange(position)[:, np.newaxis],
                           np.arange(trf_dim)[np.newaxis, :],
                           trf_dim)

    # apply sin to even indices in the array; 2i
    angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])

    # apply cos to odd indices in the array; 2i+1
    angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])

    pos_encoding = angle_rads[np.newaxis, ...]

    return tf.cast(pos_encoding, dtype=tf.float32)
    
#################################################

In [68]:
#################################################
# 2. Data Generator

class DataGenerator(Sequence):
    
    def __init__(self, list_traces, dict_traces_paths, batch_size,
                 trf_dim=128, experiment=None, shuffle=True, training=True, dict_floors=None):
        super(DataGenerator, self).__init__()
        self.list_traces = list_traces
        self.dict_traces_paths = dict_traces_paths
        self.batch_size = batch_size
        self.dict_floors = dict_floors
        self.experiment = experiment
        self.trf_dim = trf_dim
        self.training = training
        self.shuffle = shuffle
        self.pad_w = 107
        self.pad_num_w = 1_000
        self.on_epoch_end()
        
    def __len__(self):
        self.num_steps = int(np.ceil(len(self.list_traces) / self.batch_size))
        return self.num_steps
        
    def __getitem__(self, idx):
        indexes = self.indexes[idx*self.batch_size:(idx+1)*self.batch_size]
        list_batch_trace = [self.list_traces[k] for k in indexes]
        
        if self.training:
            l_arr_imu, l_arr_wifi_beacon, l_arr_wdata, l_arr_labels = [], [], [], []
            l_train_x, l_train_y, l_pad_mask = [], [], []
            for trace in list_batch_trace:
                imu = np.expand_dims(np.load(f'{self.dict_traces_paths[trace]}/seq_imu.npy', allow_pickle=True).astype(np.float32), 0)
                wifi_beacon = np.expand_dims(np.load(f'{self.dict_traces_paths[trace]}/seq_wifi_beacon.npy', allow_pickle=True).astype(np.float32), 0)
                wdata = np.expand_dims(np.load(f'{self.dict_traces_paths[trace]}/waypoint_data.npy', allow_pickle=True).astype(np.float32), 0)
                labels = np.expand_dims(np.load(f'{self.dict_traces_paths[trace]}/waypoint_predict.npy', allow_pickle=True).astype(np.float32), 0) 
                
                site, floor = wdata[:, 0, 0][0], labels[:, 0, 0][0]
                
                arr_floor = np.expand_dims(np.repeat(np.asarray([[floor + 2]]), wdata.shape[1], axis=1), -1)
                wdata = np.concatenate([arr_floor, wdata], axis=-1)
                num_w = wdata.shape[1]
                labels[:, :, 1:] /= 100.0

                padt = (0, (self.pad_w-imu.shape[1]))
                arr_imu = np.pad(imu, ((0, 0), padt, (0, 0), (0, 0)), constant_values=(0))
                arr_wifi_beacon = np.pad(wifi_beacon, ((0, 0), padt, (0, 0), (0, 0)), constant_values=(0))
                arr_wdata = np.pad(wdata, ((0, 0), padt, (0, 0)), constant_values=(0))
                arr_labels = np.pad(labels, ((0, 0), padt, (0, 0)), constant_values=(-1))
                
                arr_pad_mask = np.where(np.equal(arr_wdata[:, :, 1:], 0), 0, 1)
                arr_pad_mask[:, 0, :] = 1.
                
                arr_imu = (arr_imu - self.experiment.mean_imu)/self.experiment.std_imu
                arr_wdata[:, :, 2:] = (arr_wdata[:, :, 2:] - self.experiment.mean_w)/self.experiment.std_w
                arr_wifi_beacon[:, :, :, 2:6] = (arr_wifi_beacon[:, :, :, 2:6] - self.experiment.mean_wifi)/self.experiment.std_wifi
                arr_wifi_beacon[:, :, :, 7:] = (arr_wifi_beacon[:, :, :, 7:] - self.experiment.mean_beacon)/self.experiment.std_beacon
                
                
                l_arr_imu.append(arr_imu)
                l_arr_wifi_beacon.append(arr_wifi_beacon)
                l_arr_wdata.append(arr_wdata)
                l_arr_labels.append(arr_labels)
                l_pad_mask.append(arr_pad_mask)

            x_imu = np.concatenate(l_arr_imu)
            x_wb = np.concatenate(l_arr_wifi_beacon)
            x_wdata = np.concatenate(l_arr_wdata)

            y = np.concatenate(l_arr_labels)[:, :, 1:]
            pad_mask = np.concatenate(l_pad_mask).astype(np.float32)
            enc_pad_mask, combined_mask, _ = self.createMasks(pad_mask, x_wdata)
            pos_encoding = positionalEncoding(self.pad_w, self.trf_dim)
            # pos_encoding_beacon = 
            pos_encoding_timestamp = np.repeat(np.expand_dims(x_wdata[:, :, 3], -1), self.trf_dim, -1)
            pos_encoding_imu = np.expand_dims(positionalEncoding(128, self.trf_dim), 1)
            x_wifi = x_wb[:, :, :100, :6]
            x_beacon = x_wb[:, :, :100, 6:]
            return (x_imu, x_wifi, x_beacon, x_wdata, enc_pad_mask, pos_encoding, combined_mask), y
        
        else:
            l_arr_imu, l_arr_wifi_beacon, l_arr_wdata, l_pad_mask = [], [], [], []
            for trace in list_batch_trace:
                imu = np.expand_dims(np.load(f'{self.dict_traces_paths[trace]}/seq_imu.npy', allow_pickle=True).astype(np.float32), 0)
                wifi_beacon = np.expand_dims(np.load(f'{self.dict_traces_paths[trace]}/seq_wifi_beacon.npy', allow_pickle=True).astype(np.float32), 0)
                wdata = np.expand_dims(np.load(f'{self.dict_traces_paths[trace]}/waypoint_data.npy', allow_pickle=True).astype(np.float32), 0)
                
                site, floor = wdata[:, 0, 0][0], self.dict_floors[trace]
                arr_floor = np.expand_dims(np.repeat(np.asarray([[floor + 2]]), wdata.shape[1], axis=1), -1)
                wdata = np.concatenate([arr_floor, wdata], axis=-1)
                padt = (0, (self.pad_w-imu.shape[1]))
                arr_imu = np.pad(imu, ((0, 0), padt, (0, 0), (0, 0)), constant_values=(0))
                arr_wifi_beacon = np.pad(wifi_beacon, ((0, 0), padt, (0, 0), (0, 0)), constant_values=(0))
                arr_wdata = np.pad(wdata, ((0, 0), padt, (0, 0)), constant_values=(0))
                
                arr_pad_mask = np.where(np.equal(arr_wdata[:, :, 1:], 0), 0, 1)
                arr_pad_mask[:, 0, :] = 1.
                arr_imu = (arr_imu - self.experiment.mean_imu)/self.experiment.std_imu
                arr_wdata[:, :, 2:] = (arr_wdata[:, :, 2:] - self.experiment.mean_w)/self.experiment.std_w
                arr_wifi_beacon[:, :, :, 2:6] = (arr_wifi_beacon[:, :, :, 2:6] - self.experiment.mean_wifi)/self.experiment.std_wifi
                arr_wifi_beacon[:, :, :, 7:] = (arr_wifi_beacon[:, :, :, 7:] - self.experiment.mean_beacon)/self.experiment.std_beacon

                l_arr_imu.append(arr_imu)
                l_arr_wifi_beacon.append(arr_wifi_beacon)
                l_arr_wdata.append(arr_wdata)
                l_pad_mask.append(arr_pad_mask)
                
            x_imu = np.concatenate(l_arr_imu)
            x_wb = np.concatenate(l_arr_wifi_beacon)
            x_wdata = np.concatenate(l_arr_wdata)
            pad_mask = np.concatenate(l_pad_mask).astype(np.float32)
            enc_pad_mask, combined_mask, _ = self.createMasks(pad_mask, x_wdata)
            pos_encoding = positionalEncoding(self.pad_w, self.trf_dim)
            pos_encoding_imu = np.expand_dims(positionalEncoding(128, self.trf_dim), 1)
            pos_encoding_timestamp = np.repeat(np.expand_dims(x_wdata[:, :, 3], -1), self.trf_dim, -1)
            x_wifi = x_wb[:, :, :100, :6]
            x_beacon = x_wb[:, :, :100, 6:]
            return (x_imu, x_wifi, x_beacon, x_wdata, pad_mask, pos_encoding, combined_mask)
            
            
    def on_epoch_end(self):
        self.indexes = np.arange(len(self.list_traces))
        if self.shuffle:
            np.random.shuffle(self.list_traces)
            
    def createPaddingMask(self, x):
        mask = tf.cast(tf.math.equal(x, 1), tf.bool)
        # add extra dimensions to add the padding to the attention logits.
        # (batch_size, 1, 1, sequence length)
        mask = mask[:, tf.newaxis, tf.newaxis, :, 0]
        return mask
    
    def causal_attention_mask(self, batch_size, n_dest, n_src, dtype):
        """Masks the upper half of the dot product matrix in self attention.

        This prevents flow of information from future tokens to current token.
        1's in the lower triangle, counting from the lower right corner.
        """
        i = tf.range(n_dest)[:, None]
        j = tf.range(n_src)
        m = i >= j - n_src + n_dest
        mask = tf.cast(m, dtype)
        mask = tf.reshape(mask, [1, n_dest, n_src])
        mult = tf.concat(
            [tf.expand_dims(batch_size, -1), tf.constant([1, 1], dtype=tf.int32)], 0
        )
        return tf.tile(mask, mult)
            
    def createMasks(self, inp, tar):
        # Encoder padding mask
        enc_padding_mask = self.createPaddingMask(inp)

        # Used in the 2nd attention block in the decoder.
        # This padding mask is used to mask the encoder outputs.
        dec_padding_mask = self.createPaddingMask(inp)

        # Used in the 1st attention block in the decoder.
        # It is used to pad and mask future tokens in the input received by 
        # the decoder.
        #look_ahead_mask = self.createLookAheadMask(tf.shape(tar)[1])
        #dec_target_padding_mask = self.createPaddingMask(tar)
        #combined_mask = tf.maximum(dec_target_padding_mask, look_ahead_mask)
        combined_mask = self.causal_attention_mask(tf.shape(tar)[0], tf.shape(tar)[1], tf.shape(tar)[1], tf.bool)

        return enc_padding_mask, combined_mask, dec_padding_mask
            
            
    def createLookAheadMask(self, seq_len):
        mask = 1 - tf.linalg.band_part(tf.ones((seq_len, seq_len)), -1, 0)
        return mask
    

#################################################

In [13]:
# datagen = DataGenerator(list_train_traces, dict_train_traces_paths, batch_size=8,
#                         experiment=experiment, 
#                         shuffle=True, training=True)
# for batch in datagen:
#     break

# # x_imu, x_wifi, x_beacon, x_wdata, enc_pad_mask, pos_encoding, combined_mask
# in_imu, in_wifi, in_beacon, in_w, in_enc_pad_mask, in_pos_encoding, in_combined_mask, in_target = \
#             batch[0][0], batch[0][1], batch[0][2], batch[0][3], batch[0][4], batch[0][5], batch[0][6], batch[1]

# print(in_imu.shape, in_wifi.shape, in_beacon.shape, in_w.shape, )
# print(in_enc_pad_mask.shape, in_pos_encoding.shape, in_combined_mask.shape)
# print(in_target.shape)



In [48]:
#################################################
# 4. Models


class EncoderTransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, attention_axes=None, rate=0.1):
        super(EncoderTransformerBlock, self).__init__()
        self.att = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim, attention_axes=attention_axes)
        self.ffn = models.Sequential(
            [layers.Dense(ff_dim, activation="relu"), 
             layers.Dense(embed_dim)]
        )
        
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, query, key, training, attention_mask=None):
        attn_output = self.att(query, key, attention_mask=attention_mask)
        attn_output = self.dropout1(attn_output, training=training)
        
        out1 = self.layernorm1(query + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        
        return self.layernorm2(out1 + ffn_output)
    

class ProcessEncoderInput(layers.Layer):
    def __init__(self, trf_dim, hide_dim=50):
        super(ProcessEncoderInput, self).__init__()
        
        # Class Embedings
        self.emb_ssid = layers.Embedding(EMB_SSID_DIM, 60, mask_zero=True, name='emb_ssid')
        self.emb_bssid = layers.Embedding(EMB_BSSID_DIM, 120, mask_zero=True, name='emb_bssid')
        self.emb_uuid = layers.Embedding(EMB_UUID_DIM, 60, mask_zero=True, name='emb_uuiid')
        self.emb_site = layers.Embedding(EMB_SITE_DIM, 20, mask_zero=False, name='site_embedding')
        self.emb_floor = layers.Embedding(NUM_FLOORS, 3, mask_zero=False, name='floor_embedding')
        
        self.encoder_imu = EncoderTransformerBlock(trf_dim, num_heads, trf_dim*4)
        self.encoder_wifi = EncoderTransformerBlock(trf_dim, num_heads, trf_dim*4)
        self.encoder_beacon = EncoderTransformerBlock(trf_dim, num_heads, trf_dim*4)
        # self.encoder_waypoint = EncoderTransformerBlock(trf_dim, num_heads, trf_dim*4)
        
        # Continous features
        self.d_imu = layers.Dense(trf_dim)
        self.d_wifi_beacon = layers.Dense(trf_dim)
        self.d_way = layers.Dense(trf_dim)
        self.l_norm = layers.LayerNormalization()
        # self.d_all = layers.Dense(trf_dim)
        self.drop1 = layers.Dropout(0.2)
        self.drop2 = layers.Dropout(0.2)
        self.drop3 = layers.Dropout(0.2)
        self.drop4 = layers.Dropout(0.2)
        
        # memories
        self.seq_imu = models.Sequential([
            layers.Dropout(0.3),
            layers.Dense(64, 'relu'),
            layers.Dropout(0.2),
            layers.Dense(trf_dim),
            layers.LayerNormalization()
        ])
        
        self.seq_wifi = models.Sequential([
            layers.Dropout(0.3),
            layers.Dense(trf_dim*3, 'relu'),
            layers.Dropout(0.2),
            layers.Dense(trf_dim),
            layers.LayerNormalization()
        ])
        
        self.seq_beacon = models.Sequential([
            # layers.Dropout(0.3),
            #layers.Dense(trf_dim, 'relu'),
            layers.Dropout(0.2),
            layers.Dense(trf_dim),
            layers.LayerNormalization()
        ])
        
#         self.seq_way = models.Sequential([
#             layers.Dropout(0.3),
#             layers.Dense(20, 'relu'),
#             layers.Dropout(0.2),
#             layers.Dense(trf_dim),
#             layers.LayerNormalization()
#         ])
    
        

    def call(self, inputs, training=True):
        in_imu, in_wifi, in_beacon, in_waypoint, pos_encoding, enc_pad_mask = inputs
        
        seq_len = tf.shape(in_waypoint)[1]
        # Entities
        x_ssid = self.emb_ssid(in_wifi[:, :, :, 0])
        x_bssid = self.emb_bssid(in_wifi[:, :, :, 1])
        x_uuid = self.emb_uuid(in_beacon[:, :, :, 0])
        x_floor = self.emb_floor(in_waypoint[:, :, 0])
        x_site = self.emb_site(in_waypoint[:, :, 1])
        
        x_imu = tf.reshape(in_imu, [tf.shape(in_imu)[0], tf.shape(in_imu)[1], tf.shape(in_imu)[2]*tf.shape(in_imu)[3]])
        x_imu = self.seq_imu(x_imu)
        
        x_wifi = tf.concat([x_ssid, x_bssid, in_wifi[:, :, :, 2:6]], axis=-1)
        x_wifi = tf.reshape(x_wifi, [tf.shape(x_wifi)[0], tf.shape(x_wifi)[1], 
                                                 tf.shape(x_wifi)[2]*tf.shape(x_wifi)[3]])
        x_wifi = self.seq_wifi(x_wifi)
        
        x_beacon = tf.concat([x_uuid, in_beacon[:, :, :, 1:]], axis=-1)
        x_beacon = tf.reshape(x_beacon, [tf.shape(x_beacon)[0], tf.shape(x_beacon)[1], 
                                         tf.shape(x_beacon)[2]*tf.shape(x_beacon)[3]])
        x_beacon = self.seq_beacon(x_beacon)
        
        x_way = tf.concat([x_floor, x_site, in_waypoint[:, :, 2:]], axis=-1)
        # x_way = self.seq_way(x_way)
        
        # x = x_imu + x_wifi + x_beacon + x_way + pos_encoding
        x_imu = self.encoder_imu(x_imu, x_imu, training=training, attention_mask=enc_pad_mask)
        x_wifi = self.encoder_wifi(x_wifi, x_wifi, training=training, attention_mask=enc_pad_mask)
        x_beacon = self.encoder_beacon(x_beacon, x_beacon, training=training, attention_mask=enc_pad_mask)
        # x_way = self.encoder_waypoint(x_way, x_way, training=training, attention_mask=enc_pad_mask)
        
        x =  x_imu + x_wifi + x_beacon# + x_way
        x = self.l_norm(x)
        
        return x, x_way
    
    
    
class Encoder(models.Model):
    def __init__(self, trf_dim=64, num_heads=2, ff_dim=100, num_layers=1):
        super(Encoder, self).__init__()
        self.trf_dim = trf_dim
        self.num_layers = num_layers
        self.d_dim = layers.Dense(trf_dim)
        self.dropout = layers.Dropout(0.1)
        self.list_transformer_blocks = [EncoderTransformerBlock(trf_dim, num_heads, ff_dim)
                                           for _ in range(num_layers)]
        
        self.gru = layers.GRU(trf_dim, return_sequences=True, dropout=0.1)
    
    def call(self, inputs, training=True):
        enc_input, pad_mask, pos_encoding, in_combined_mask = inputs
        seq_len = tf.shape(enc_input)[1]

        x = self.dropout(enc_input, training=training)
        
        for i in range(self.num_layers):
            x = self.list_transformer_blocks[i](x, x, training=training, attention_mask=pad_mask)
        return x
    
    

class TransformerModel(models.Model):
    
    def __init__(self, hide_dim, trf_dim, num_heads, ff_dim, num_enc_layers, max_seq_len):
        super(TransformerModel, self).__init__()
        self.trf_dim = trf_dim
        self.proc_encoder = ProcessEncoderInput(trf_dim=trf_dim, hide_dim=hide_dim)
        self.encoder = Encoder(trf_dim=trf_dim, num_heads=num_heads, ff_dim=trf_dim*4, num_layers=num_enc_layers)
        ###
        self.gru = layers.Bidirectional(layers.GRU(trf_dim//4, return_sequences=True, dropout=0.3))
        self.drop_out_1 = layers.Dropout(0.1)
        self.drop_out_2 = layers.Dropout(0.1)
        self.dense_out1 = layers.Dense(32, activation='relu')
        self.dense_out2 = layers.Dense(2, activation='relu')
    
    def call(self, inputs, training=True):
        ###
        in_imu, in_wifi, in_beacon, in_way, in_enc_pad_mask, pos_encoding, in_combined_mask = inputs
        seq_len = tf.shape(in_way)[1]
        ### 
        ###
        p_enc_out, x_sf = self.proc_encoder((in_imu, in_wifi, in_beacon, in_way, pos_encoding, in_enc_pad_mask), training=training)
        
        enc_output = self.encoder((p_enc_out, in_enc_pad_mask, pos_encoding, in_combined_mask), 
                                  training=training)
        # print(f'Enc output: {enc_output.shape}')
        out = self.gru(enc_output, 
                       training=training, 
                       mask=tf.cast(in_enc_pad_mask[:, 0, 0, :seq_len], tf.bool))

        out = self.drop_out_1(out, training=training)
        out = tf.concat([out, x_sf], axis=-1)
        out = self.dense_out1(out)
        # out = self.drop_out_2(out, training=training)
        out = self.dense_out2(out)  
        
        return out

    
    
def getTransformerModel(hide_dim, trf_dim, num_heads, ff_dim, num_enc_layers, max_seq_len):
    return TransformerModel(hide_dim, trf_dim, num_heads, ff_dim, num_enc_layers, max_seq_len)


def lossFunction(y_real, y_pred):
    mask = tf.math.logical_not(tf.math.equal(y_real, -1))
    weights = tf.concat([tf.ones((1, 1, 1)) * 2.0, tf.ones((1, tf.shape(y_real)[1]-1, 1))], axis=1)
    loss_ = tf.math.sqrt(tf.expand_dims(tf.square(tf.math.subtract(y_real[:, :, 0], y_pred[:, :, 0])), -1) + \
                         tf.expand_dims(tf.square(tf.math.subtract(y_real[:, :, 1], y_pred[:, :, 1])), -1))

    mask = tf.cast(mask, dtype=loss_.dtype)
    mask = tf.expand_dims(tf.reduce_max(mask, axis=-1), -1)
    loss_ *= mask
    loss_ *= weights
    
    return  tf.reduce_sum(loss_)/tf.reduce_sum(mask)

def cMetric(y_real, y_pred):
    mask = tf.math.logical_not(tf.math.equal(y_real, -1))
    loss_ = tf.math.sqrt(tf.expand_dims(tf.square(tf.math.subtract(y_real[:, :, 0], y_pred[:, :, 0])), -1) + \
                         tf.expand_dims(tf.square(tf.math.subtract(y_real[:, :, 1], y_pred[:, :, 1])), -1))

    mask = tf.cast(mask, dtype=loss_.dtype)
    mask = tf.expand_dims(tf.reduce_max(mask, axis=-1), -1)
    loss_ *= mask
    
    return  tf.reduce_sum(loss_)/tf.reduce_sum(mask)

def maskedMAE(y_real, y_pred):
    mask = tf.math.logical_not(tf.math.equal(y_real, -1))
    metric = tf.math.abs(tf.math.subtract(y_real, y_pred))

    mask = tf.cast(mask, dtype=metric.dtype)
    metric *= mask

    return tf.reduce_sum(metric)/tf.reduce_sum(mask)

def maskedMAEStart(y_real, y_pred):
    mask = tf.math.logical_not(tf.math.equal(y_real[:, 0, :], -1))
    metric = tf.math.abs(tf.math.subtract(y_real[:, 0, :], y_pred[:, 0, :]))

    mask = tf.cast(mask, dtype=metric.dtype)
    metric *= mask

    return tf.reduce_sum(metric)/tf.reduce_sum(mask)
    
def plotMetrics(history, figsize=(12, 8)):
    # Accuracy
    plt.figure(figsize=figsize)
    plt.plot(history.history['maskedMAE'])
    plt.plot(history.history['val_maskedMAE'])
    plt.title('Model Masked MAE')
    plt.ylabel('maskedMAE')
    plt.xlabel('epoch')
    plt.legend(['train', 'val'], loc='upper left')
    plt.show()
    
    # Loss
    plt.figure(figsize=figsize)
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Model Loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'val'], loc='upper left')
    plt.show()


class ReturnBestEarlyStopping(tf.keras.callbacks.EarlyStopping):
    def __init__(self, **kwargs):
        super(ReturnBestEarlyStopping, self).__init__(**kwargs)

    def on_train_end(self, logs=None):
        if self.stopped_epoch > 0:
            if self.verbose > 0:
                print(f'\nEpoch {self.stopped_epoch + 1}: early stopping')
        elif self.restore_best_weights:
            if self.verbose > 0:
                print('Restoring model weights from the end of the best epoch.')
            self.model.set_weights(self.best_weights)
            
            
class CustomSchedule(optimizers.schedules.LearningRateSchedule):
    def __init__(self, d_model, warmup_steps=4000):
        super(CustomSchedule, self).__init__()

        self.d_model = d_model
        self.d_model = tf.cast(self.d_model, tf.float32)

        self.warmup_steps = warmup_steps

    def __call__(self, step):
        arg1 = tf.math.rsqrt(step)
        arg2 = step * (self.warmup_steps ** -1.5)

        return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)

                  
def scheduler(epoch, lr):
    if epoch > 20 and epoch % 5 == 0:
        return lr*0.9
    else:
        return lr
                                   
#################################################

In [49]:
# def createPaddingMask(x):
#     mask = tf.cast(tf.math.equal(x, 0), tf.float32)
#     # add extra dimensions to add the padding to the attention logits.
#     # (batch_size, 1, 1, sequence length)
#     mask = mask[:, tf.newaxis, tf.newaxis, :, 0]
#     return mask

# datagen = DataGenerator(list_train_traces, dict_train_traces_paths, batch_size=8, trf_dim=128,
#                                      experiment=experiment, shuffle=True, training=True)
# for batch in datagen:
#     break

# print(len(batch[0]), len(batch[0][0]), len(batch[0][1]), len(batch[0][2]), len(batch[1]))
# print(batch[0][0].shape, batch[0][1].shape, batch[0][2].shape, batch[1].shape)

# in_imu, in_wifibeacon, in_w, in_shift_target, in_enc_pad_mask, in_dec_pad_mask, in_look_ahead_mask, \
#          in_pos_encoding, in_target = \
#             batch[0][0], batch[0][1], batch[0][2], batch[0][3], batch[0][4], batch[0][5], \
#             batch[0][6], batch[0][7], batch[1]

# print(in_imu.shape, in_wifibeacon.shape, in_w.shape, in_shift_target.shape, in_enc_pad_mask.shape, in_dec_pad_mask.shape,
#       in_look_ahead_mask.shape, in_pos_encoding.shape, in_target.shape)


# hide_dim = 32
# trf_dim = 128
# num_heads = 2

# pos_encoding = np.zeros((1, 107, trf_dim))
# enc_pad_mask = createPaddingMask(in_enc_pad_mask)
# dec_pad_mask = createPaddingMask(in_dec_pad_mask)

# p_enc = ProcessEncoderInput(trf_dim=trf_dim, hide_dim=hide_dim)
# p_enc_out = p_enc((in_imu, in_wifibeacon))
# print(f'ProcessEncoderInput out: {p_enc_out.shape}')

# enc = Encoder(trf_dim=trf_dim, num_heads=num_heads, ff_dim=trf_dim*4)
# enc_output = enc((p_enc_out, enc_pad_mask, in_pos_encoding))
# print(f'Encoder out: {enc_output.shape}')

# p_dec = ProcessDecoderInput(trf_dim=trf_dim)
# p_dec_out = p_dec((in_w, in_shift_target))
# print(f'ProcessDecoderInput out: {p_dec_out.shape}')

# dec = Decoder(trf_dim=trf_dim, num_heads=num_heads, ff_dim=trf_dim*4)
# dec_output = dec((enc_output, p_dec_out, in_pos_encoding, dec_pad_mask, in_look_ahead_mask))
# print(f'Decoder: {dec_output.shape}')

# m = TransformerModel(hide_dim=hide_dim, trf_dim=trf_dim, num_heads=num_heads, ff_dim=trf_dim*4)
# m_output = m((in_imu, in_wifibeacon, in_w, in_shift_target, in_enc_pad_mask, in_dec_pad_mask, in_look_ahead_mask))
# print(f'Transformer model: {m_output.shape}')

# 7 8 8 8 8
# (8, 107, 128, 23) (8, 107, 128, 8) (8, 107, 3) (8, 107, 2)
# (8, 107, 128, 23) (8, 107, 128, 8) (8, 107, 3) (8, 107, 2) (8, 107, 2) (8, 107, 2) (8, 107, 107) (8, 107, 2)
# ProcessEncoderInput out: (8, 107, 15872)
# Encoder out: (8, 107, 128)
# ProcessDecoderInput out: (8, 107, 128)
# Decoder: (8, 107, 128)
# Transformer model: (8, 107, 2)

In [50]:
# val_data_generator = DataGenerator(['5dc23ce006f7170006addcc0'], dict_train_traces_paths, batch_size=1, 
#                                    experiment=experiment, shuffle=False, training=True)

# for batch in val_data_generator:
#     break
    
# predictions = model(batch[0], training=False)
# target = batch[1]

# print(target)
# print(predictions)

In [51]:
#################################################
# 4. Model training

callback_early_stopping = ReturnBestEarlyStopping(monitor='val_cMetric', 
                                                  patience=20, verbose=1, restore_best_weights=True)
callback_lrsched = tf.keras.callbacks.LearningRateScheduler(scheduler)
list_callbacks = [callback_early_stopping, callback_lrsched]

learning_rate = 1e-3
hide_dim = 30
trf_dim = 128
num_enc_layers = 1
num_heads = 2
batch_size = 64
val_size = 0.1
model_name = 'trf_enc_dec_v0.13'

print(f'Num train paths: {len(list_train_train_traces)}, Num val paths: {len(list_train_val_traces)}')
assert len(list_train_train_traces)+len(list_train_val_traces)==len(list_train_traces)

train_data_generator = DataGenerator(list_train_train_traces, dict_train_traces_paths, batch_size=batch_size,
                                     trf_dim=trf_dim, experiment=experiment, shuffle=True, training=True)

val_data_generator = DataGenerator(list_train_val_traces, dict_train_traces_paths, batch_size=batch_size, 
                                   trf_dim=trf_dim, experiment=experiment, shuffle=True, training=True)

# Model

model = getTransformerModel(hide_dim=hide_dim, trf_dim=trf_dim, num_heads=num_heads, ff_dim=trf_dim*4,
                            num_enc_layers=num_enc_layers, max_seq_len=107)
# Loads the weights
# model.load_weights(PATH_MODELS + 'position_model/' + model_name + '_weights')
learning_rate = 1e-3
# learning_rate = CustomSchedule(trf_dim)
model.compile(optimizer=optimizers.Adam(learning_rate=learning_rate, beta_1=0.9, beta_2=0.98, 
                                        epsilon=1e-9), 
              loss=cMetric,
              metrics=[cMetric, maskedMAE, maskedMAEStart])

history = model.fit(train_data_generator,
                    validation_data=val_data_generator,
                    batch_size=batch_size,
                    callbacks=list_callbacks,
                    epochs=100,
                    verbose=1)

plotMetrics(history, figsize=(10, 6)) 
model.save(PATH_MODELS + 'position_model/' + model_name, include_optimizer=False)
model.save_weights(PATH_MODELS + 'position_model/' + model_name + '_weights', save_format='tf')

#################################################

Num train paths: 23620, Num val paths: 844
Epoch 1/100
370/370 [==============================] - 220s 570ms/step - loss: 0.7131 - cMetric: 0.7138 - maskedMAE: 0.4528 - maskedMAEStart: 0.4316 - val_loss: 0.2805 - val_cMetric: 0.2810 - val_maskedMAE: 0.1778 - val_maskedMAEStart: 0.1785
Epoch 2/100
370/370 [==============================] - 210s 566ms/step - loss: 0.2662 - cMetric: 0.2662 - maskedMAE: 0.1690 - maskedMAEStart: 0.1621 - val_loss: 0.1970 - val_cMetric: 0.2108 - val_maskedMAE: 0.1342 - val_maskedMAEStart: 0.1237
Epoch 3/100
370/370 [==============================] - 208s 560ms/step - loss: 0.2010 - cMetric: 0.2010 - maskedMAE: 0.1278 - maskedMAEStart: 0.1208 - val_loss: 0.1885 - val_cMetric: 0.1873 - val_maskedMAE: 0.1179 - val_maskedMAEStart: 0.1112
Epoch 4/100
370/370 [==============================] - 214s 577ms/step - loss: 0.1672 - cMetric: 0.1671 - maskedMAE: 0.1062 - maskedMAEStart: 0.1039 - val_loss: 0.1571 - val_cMetric: 0.1575 - val_maskedMAE: 0.1001 - val_maskedMA

KeyboardInterrupt: 

In [ ]:
# Num train paths: 23620, Num val paths: 844
# Epoch 1/100
# 370/370 [==============================] - 220s 570ms/step - loss: 0.7131 - cMetric: 0.7138 - maskedMAE: 0.4528 - maskedMAEStart: 0.4316 - val_loss: 0.2805 - val_cMetric: 0.2810 - val_maskedMAE: 0.1778 - val_maskedMAEStart: 0.1785
# Epoch 2/100
# 370/370 [==============================] - 210s 566ms/step - loss: 0.2662 - cMetric: 0.2662 - maskedMAE: 0.1690 - maskedMAEStart: 0.1621 - val_loss: 0.1970 - val_cMetric: 0.2108 - val_maskedMAE: 0.1342 - val_maskedMAEStart: 0.1237
# Epoch 3/100
# 370/370 [==============================] - 208s 560ms/step - loss: 0.2010 - cMetric: 0.2010 - maskedMAE: 0.1278 - maskedMAEStart: 0.1208 - val_loss: 0.1885 - val_cMetric: 0.1873 - val_maskedMAE: 0.1179 - val_maskedMAEStart: 0.1112
# Epoch 4/100
# 370/370 [==============================] - 214s 577ms/step - loss: 0.1672 - cMetric: 0.1671 - maskedMAE: 0.1062 - maskedMAEStart: 0.1039 - val_loss: 0.1571 - val_cMetric: 0.1575 - val_maskedMAE: 0.1001 - val_maskedMAEStart: 0.0982
# Epoch 5/100
# 370/370 [==============================] - 211s 569ms/step - loss: 0.1487 - cMetric: 0.1486 - maskedMAE: 0.0944 - maskedMAEStart: 0.0926 - val_loss: 0.1476 - val_cMetric: 0.1500 - val_maskedMAE: 0.0949 - val_maskedMAEStart: 0.0876
# Epoch 6/100
# 370/370 [==============================] - 212s 572ms/step - loss: 0.1359 - cMetric: 0.1359 - maskedMAE: 0.0863 - maskedMAEStart: 0.0845 - val_loss: 0.1297 - val_cMetric: 0.1286 - val_maskedMAE: 0.0814 - val_maskedMAEStart: 0.0846
# Epoch 7/100
# 370/370 [==============================] - 207s 559ms/step - loss: 0.1263 - cMetric: 0.1266 - maskedMAE: 0.0804 - maskedMAEStart: 0.0797 - val_loss: 0.1268 - val_cMetric: 0.1280 - val_maskedMAE: 0.0815 - val_maskedMAEStart: 0.0776
# Epoch 8/100
# 370/370 [==============================] - 205s 553ms/step - loss: 0.1182 - cMetric: 0.1182 - maskedMAE: 0.0752 - maskedMAEStart: 0.0746 - val_loss: 0.1246 - val_cMetric: 0.1235 - val_maskedMAE: 0.0787 - val_maskedMAEStart: 0.0885
# Epoch 9/100
# 370/370 [==============================] - 205s 554ms/step - loss: 0.1128 - cMetric: 0.1127 - maskedMAE: 0.0716 - maskedMAEStart: 0.0714 - val_loss: 0.1132 - val_cMetric: 0.1125 - val_maskedMAE: 0.0710 - val_maskedMAEStart: 0.0736
# Epoch 10/100
# 370/370 [==============================] - 208s 560ms/step - loss: 0.1073 - cMetric: 0.1074 - maskedMAE: 0.0684 - maskedMAEStart: 0.0674 - val_loss: 0.1135 - val_cMetric: 0.1151 - val_maskedMAE: 0.0731 - val_maskedMAEStart: 0.0764
# Epoch 11/100
# 370/370 [==============================] - 209s 564ms/step - loss: 0.1024 - cMetric: 0.1023 - maskedMAE: 0.0649 - maskedMAEStart: 0.0644 - val_loss: 0.1134 - val_cMetric: 0.1143 - val_maskedMAE: 0.0725 - val_maskedMAEStart: 0.0714
# Epoch 12/100
# 370/370 [==============================] - 211s 569ms/step - loss: 0.1011 - cMetric: 0.1011 - maskedMAE: 0.0643 - maskedMAEStart: 0.0632 - val_loss: 0.1098 - val_cMetric: 0.1093 - val_maskedMAE: 0.0693 - val_maskedMAEStart: 0.0707
# Epoch 13/100
# 370/370 [==============================] - 210s 566ms/step - loss: 0.0960 - cMetric: 0.0960 - maskedMAE: 0.0609 - maskedMAEStart: 0.0604 - val_loss: 0.1046 - val_cMetric: 0.1055 - val_maskedMAE: 0.0672 - val_maskedMAEStart: 0.0703
# Epoch 14/100
# 370/370 [==============================] - 211s 569ms/step - loss: 0.0933 - cMetric: 0.0931 - maskedMAE: 0.0591 - maskedMAEStart: 0.0582 - val_loss: 0.1001 - val_cMetric: 0.0992 - val_maskedMAE: 0.0628 - val_maskedMAEStart: 0.0651
# Epoch 15/100
# 370/370 [==============================] - 213s 574ms/step - loss: 0.0904 - cMetric: 0.0904 - maskedMAE: 0.0574 - maskedMAEStart: 0.0573 - val_loss: 0.0990 - val_cMetric: 0.0989 - val_maskedMAE: 0.0629 - val_maskedMAEStart: 0.0658
# Epoch 16/100
# 370/370 [==============================] - 216s 584ms/step - loss: 0.0883 - cMetric: 0.0882 - maskedMAE: 0.0561 - maskedMAEStart: 0.0554 - val_loss: 0.0989 - val_cMetric: 0.0989 - val_maskedMAE: 0.0629 - val_maskedMAEStart: 0.0675
# Epoch 17/100
# 370/370 [==============================] - 212s 573ms/step - loss: 0.0864 - cMetric: 0.0864 - maskedMAE: 0.0549 - maskedMAEStart: 0.0544 - val_loss: 0.0998 - val_cMetric: 0.0994 - val_maskedMAE: 0.0632 - val_maskedMAEStart: 0.0625
# Epoch 18/100
# 370/370 [==============================] - 213s 574ms/step - loss: 0.0837 - cMetric: 0.0837 - maskedMAE: 0.0532 - maskedMAEStart: 0.0521 - val_loss: 0.1015 - val_cMetric: 0.1008 - val_maskedMAE: 0.0643 - val_maskedMAEStart: 0.0650
# Epoch 19/100
# 370/370 [==============================] - 216s 584ms/step - loss: 0.0831 - cMetric: 0.0837 - maskedMAE: 0.0532 - maskedMAEStart: 0.0518 - val_loss: 0.0978 - val_cMetric: 0.0979 - val_maskedMAE: 0.0617 - val_maskedMAEStart: 0.0652
# Epoch 20/100
# 370/370 [==============================] - 214s 577ms/step - loss: 0.0806 - cMetric: 0.0806 - maskedMAE: 0.0512 - maskedMAEStart: 0.0502 - val_loss: 0.0979 - val_cMetric: 0.1021 - val_maskedMAE: 0.0649 - val_maskedMAEStart: 0.0642
# Epoch 21/100
# 370/370 [==============================] - 216s 583ms/step - loss: 0.0795 - cMetric: 0.0796 - maskedMAE: 0.0505 - maskedMAEStart: 0.0497 - val_loss: 0.1028 - val_cMetric: 0.1012 - val_maskedMAE: 0.0644 - val_maskedMAEStart: 0.0639
# Epoch 22/100
# 370/370 [==============================] - 218s 589ms/step - loss: 0.0774 - cMetric: 0.0774 - maskedMAE: 0.0491 - maskedMAEStart: 0.0487 - val_loss: 0.1084 - val_cMetric: 0.1083 - val_maskedMAE: 0.0686 - val_maskedMAEStart: 0.0709
# Epoch 23/100
# 370/370 [==============================] - 263s 710ms/step - loss: 0.0754 - cMetric: 0.0753 - maskedMAE: 0.0478 - maskedMAEStart: 0.0479 - val_loss: 0.0985 - val_cMetric: 0.0976 - val_maskedMAE: 0.0617 - val_maskedMAEStart: 0.0620
# Epoch 24/100
# 370/370 [==============================] - 408s 1s/step - loss: 0.0745 - cMetric: 0.0745 - maskedMAE: 0.0473 - maskedMAEStart: 0.0471 - val_loss: 0.0909 - val_cMetric: 0.0910 - val_maskedMAE: 0.0581 - val_maskedMAEStart: 0.0595
# Epoch 25/100
# 370/370 [==============================] - 336s 907ms/step - loss: 0.0720 - cMetric: 0.0720 - maskedMAE: 0.0457 - maskedMAEStart: 0.0457 - val_loss: 0.0915 - val_cMetric: 0.0910 - val_maskedMAE: 0.0580 - val_maskedMAEStart: 0.0588
# Epoch 26/100
# 370/370 [==============================] - 652s 2s/step - loss: 0.0722 - cMetric: 0.0722 - maskedMAE: 0.0459 - maskedMAEStart: 0.0452 - val_loss: 0.0899 - val_cMetric: 0.0885 - val_maskedMAE: 0.0561 - val_maskedMAEStart: 0.0578
# Epoch 27/100
# 370/370 [==============================] - 664s 2s/step - loss: 0.0698 - cMetric: 0.0698 - maskedMAE: 0.0444 - maskedMAEStart: 0.0442 - val_loss: 0.0901 - val_cMetric: 0.0890 - val_maskedMAE: 0.0565 - val_maskedMAEStart: 0.0602
# Epoch 28/100
# 370/370 [==============================] - 667s 2s/step - loss: 0.0695 - cMetric: 0.0695 - maskedMAE: 0.0441 - maskedMAEStart: 0.0440 - val_loss: 0.0918 - val_cMetric: 0.0906 - val_maskedMAE: 0.0574 - val_maskedMAEStart: 0.0601
# Epoch 29/100
# 370/370 [==============================] - 549s 1s/step - loss: 0.0688 - cMetric: 0.0687 - maskedMAE: 0.0436 - maskedMAEStart: 0.0432 - val_loss: 0.0994 - val_cMetric: 0.1001 - val_maskedMAE: 0.0638 - val_maskedMAEStart: 0.0667
# Epoch 30/100
# 370/370 [==============================] - 205s 553ms/step - loss: 0.0681 - cMetric: 0.0679 - maskedMAE: 0.0431 - maskedMAEStart: 0.0433 - val_loss: 0.0908 - val_cMetric: 0.0892 - val_maskedMAE: 0.0564 - val_maskedMAEStart: 0.0602
# Epoch 31/100
# 370/370 [==============================] - 204s 551ms/step - loss: 0.0654 - cMetric: 0.0653 - maskedMAE: 0.0415 - maskedMAEStart: 0.0418 - val_loss: 0.0891 - val_cMetric: 0.0887 - val_maskedMAE: 0.0559 - val_maskedMAEStart: 0.0567
# Epoch 32/100
# 370/370 [==============================] - 207s 559ms/step - loss: 0.0664 - cMetric: 0.0663 - maskedMAE: 0.0421 - maskedMAEStart: 0.0421 - val_loss: 0.0931 - val_cMetric: 0.0924 - val_maskedMAE: 0.0582 - val_maskedMAEStart: 0.0612
# Epoch 33/100
# 370/370 [==============================] - 204s 552ms/step - loss: 0.0640 - cMetric: 0.0640 - maskedMAE: 0.0406 - maskedMAEStart: 0.0412 - val_loss: 0.0931 - val_cMetric: 0.0946 - val_maskedMAE: 0.0599 - val_maskedMAEStart: 0.0610
# Epoch 34/100
# 370/370 [==============================] - 204s 551ms/step - loss: 0.0637 - cMetric: 0.0637 - maskedMAE: 0.0404 - maskedMAEStart: 0.0405 - val_loss: 0.0906 - val_cMetric: 0.0943 - val_maskedMAE: 0.0603 - val_maskedMAEStart: 0.0609
# Epoch 35/100
# 370/370 [==============================] - 204s 551ms/step - loss: 0.0631 - cMetric: 0.0631 - maskedMAE: 0.0401 - maskedMAEStart: 0.0402 - val_loss: 0.0885 - val_cMetric: 0.0891 - val_maskedMAE: 0.0561 - val_maskedMAEStart: 0.0570
# Epoch 36/100
# 370/370 [==============================] - 204s 552ms/step - loss: 0.0624 - cMetric: 0.0624 - maskedMAE: 0.0396 - maskedMAEStart: 0.0398 - val_loss: 0.0931 - val_cMetric: 0.0919 - val_maskedMAE: 0.0574 - val_maskedMAEStart: 0.0579
# Epoch 37/100
# 370/370 [==============================] - 206s 555ms/step - loss: 0.0625 - cMetric: 0.0625 - maskedMAE: 0.0396 - maskedMAEStart: 0.0392 - val_loss: 0.0876 - val_cMetric: 0.0865 - val_maskedMAE: 0.0547 - val_maskedMAEStart: 0.0567
# Epoch 38/100
# 370/370 [==============================] - 210s 568ms/step - loss: 0.0594 - cMetric: 0.0594 - maskedMAE: 0.0377 - maskedMAEStart: 0.0376 - val_loss: 0.0889 - val_cMetric: 0.0885 - val_maskedMAE: 0.0562 - val_maskedMAEStart: 0.0573
# Epoch 39/100
# 370/370 [==============================] - 209s 565ms/step - loss: 0.0597 - cMetric: 0.0597 - maskedMAE: 0.0379 - maskedMAEStart: 0.0384 - val_loss: 0.0887 - val_cMetric: 0.0881 - val_maskedMAE: 0.0556 - val_maskedMAEStart: 0.0570
# Epoch 40/100
# 370/370 [==============================] - 209s 565ms/step - loss: 0.0596 - cMetric: 0.0596 - maskedMAE: 0.0378 - maskedMAEStart: 0.0380 - val_loss: 0.0863 - val_cMetric: 0.0860 - val_maskedMAE: 0.0546 - val_maskedMAEStart: 0.0593
# Epoch 41/100
# 370/370 [==============================] - 207s 560ms/step - loss: 0.0595 - cMetric: 0.0596 - maskedMAE: 0.0378 - maskedMAEStart: 0.0382 - val_loss: 0.0881 - val_cMetric: 0.0876 - val_maskedMAE: 0.0556 - val_maskedMAEStart: 0.0591
# Epoch 42/100
# 370/370 [==============================] - 207s 560ms/step - loss: 0.0574 - cMetric: 0.0574 - maskedMAE: 0.0364 - maskedMAEStart: 0.0367 - val_loss: 0.0853 - val_cMetric: 0.0846 - val_maskedMAE: 0.0538 - val_maskedMAEStart: 0.0562
# Epoch 43/100
# 370/370 [==============================] - 209s 565ms/step - loss: 0.0565 - cMetric: 0.0565 - maskedMAE: 0.0359 - maskedMAEStart: 0.0360 - val_loss: 0.0830 - val_cMetric: 0.0827 - val_maskedMAE: 0.0521 - val_maskedMAEStart: 0.0553
# Epoch 44/100
# 370/370 [==============================] - 209s 563ms/step - loss: 0.0571 - cMetric: 0.0570 - maskedMAE: 0.0362 - maskedMAEStart: 0.0365 - val_loss: 0.0846 - val_cMetric: 0.0848 - val_maskedMAE: 0.0542 - val_maskedMAEStart: 0.0569
# Epoch 45/100
# 370/370 [==============================] - 214s 578ms/step - loss: 0.0550 - cMetric: 0.0550 - maskedMAE: 0.0349 - maskedMAEStart: 0.0351 - val_loss: 0.0833 - val_cMetric: 0.0843 - val_maskedMAE: 0.0534 - val_maskedMAEStart: 0.0566
# Epoch 46/100
# 370/370 [==============================] - 214s 578ms/step - loss: 0.0558 - cMetric: 0.0558 - maskedMAE: 0.0354 - maskedMAEStart: 0.0357 - val_loss: 0.0841 - val_cMetric: 0.0841 - val_maskedMAE: 0.0534 - val_maskedMAEStart: 0.0557
# Epoch 47/100
# 370/370 [==============================] - 213s 574ms/step - loss: 0.0547 - cMetric: 0.0547 - maskedMAE: 0.0347 - maskedMAEStart: 0.0350 - val_loss: 0.0897 - val_cMetric: 0.0889 - val_maskedMAE: 0.0565 - val_maskedMAEStart: 0.0578
# Epoch 48/100
# 370/370 [==============================] - 214s 578ms/step - loss: 0.0543 - cMetric: 0.0543 - maskedMAE: 0.0344 - maskedMAEStart: 0.0346 - val_loss: 0.0841 - val_cMetric: 0.0861 - val_maskedMAE: 0.0544 - val_maskedMAEStart: 0.0553

In [74]:
#################################################
# Validation predictions

def evaluateTraces(batch, model, batch_size=1, max_num_waypoints=107):
    x_imu, x_wifi, x_beacon, x_wdata, pad_mask, pos_encoding, combined_mask = batch
    arr_max_lengths = (pad_mask.shape[1] - (pad_mask!=0)[:,::-1].argmax(1) - 1)[:, 0]
    enc_pad_mask, combined_mask, _ = data_generator.createMasks(pad_mask, x_wdata)
    for num_batch in range(batch_size):
        max_length = arr_max_lengths[num_batch]
        output = np.zeros((1, max_num_waypoints, 2))
        data_batch = (x_imu, 
                      x_wifi, 
                      x_beacon,
                      x_wdata, 
                      enc_pad_mask,
                      pos_encoding,
                      combined_mask)
        predictions = model(data_batch, training=False)
    return predictions.numpy().squeeze(0)[:max_length+1, :]
    

# load_model_name = 'trf_enc_dec_v0.13'
# model = models.load_model(PATH_MODELS + 'position_model/' + load_model_name, compile=False)

data_generator = DataGenerator(list_train_val_traces, dict_train_traces_paths, batch_size=1, 
                               dict_floors=dict_training_floor,
                               experiment=experiment, shuffle=False, training=False)
dict_predictions = {}
for i, batch in enumerate(tqdm(data_generator)):
    arr_predictions = evaluateTraces(batch, model, batch_size=1)
    trace = list_train_val_traces[i]
    dict_predictions[trace] = {
        'site' : dict_site_floor_traces[trace][0],
        'floor' : dict_site_floor_traces[trace][1],
        'predictions' : arr_predictions
    }
    
if not os.path.exists(PATH_PREDICTIONS + load_model_name):
    os.mkdir(PATH_PREDICTIONS + load_model_name)
    
with open(f'{PATH_PREDICTIONS}{load_model_name}/val_predictions.pkl', 'wb') as f:
    pickle.dump(dict_predictions, f)    
    

#################################################

In [77]:
#################################################
# Test predictions

# 0. Get floors
dict_all_sites = dict_all_sites = experiment.dict_all_sites
df_floor = pd.read_csv(PATH_PREDICTIONS + 'floor_model_v0.3' + '/submission.csv')
df_floor = df_floor[['site_path_timestamp', 'floor']]

df_floor['site'] = df_floor['site_path_timestamp'].\
                                              apply(lambda x: dict_all_sites[x.split('_')[0]])
df_floor['path'] = df_floor['site_path_timestamp'].\
                                apply(lambda x: x.split('_')[1])
dict_floors_test = {}
for i, row in df_floor.iterrows():
    dict_floors_test[row['path']] = row['floor']
    
df_sample_submission = pd.read_csv(PATH_DATA + 'sample_submission.csv')
df_sample_submission['site'] = df_sample_submission['site_path_timestamp'].\
                                              apply(lambda x: dict_all_sites[x.split('_')[0]])
df_sample_submission['path'] = df_sample_submission['site_path_timestamp'].\
                                apply(lambda x: x.split('_')[1])
df_sample_submission['timestamp'] = df_sample_submission['site_path_timestamp'].\
                                apply(lambda x: x.split('_')[2]).astype(np.int64)/1000.0

list_columns = ['site_path_timestamp', 'site', 'path', 'timestamp']
df_sample_submission = df_sample_submission[list_columns]

dict_all_sites_inv = { i: s for i, s in enumerate(dict_all_sites)}
dict_test_site_floor_traces = {}
for i, row in df_sample_submission.iterrows():
    dict_test_site_floor_traces[row['path']] = (dict_all_sites_inv[row['site']], dict_floors_test[row['path']])

def evaluateTraces(batch, model, batch_size=1, max_num_waypoints=107):
    x_imu, x_wifi, x_beacon, x_wdata, pad_mask, pos_encoding, pos_encoding_timestamp = batch
    arr_max_lengths = (pad_mask.shape[1] - (pad_mask!=0)[:,::-1].argmax(1) - 1)[:, 0]
    enc_pad_mask, combined_mask, _ = data_generator.createMasks(pad_mask, x_wdata)
    for num_batch in range(batch_size):
        max_length = arr_max_lengths[num_batch]
        output = np.zeros((1, max_num_waypoints, 2))
        data_batch = (x_imu, 
                      x_wifi,
                      x_beacon,
                      x_wdata, 
                      enc_pad_mask,
                      pos_encoding,
                      combined_mask)
        predictions = model(data_batch, training=False)
    return predictions.numpy().squeeze(0)[:max_length+1, :], arr_max_lengths
    

# load_model_name = 'trf_enc_dec_v0.13'
# model = models.load_model(PATH_MODELS + 'position_model/' + load_model_name, compile=False)

test_data_generator = DataGenerator(list_test_traces, dict_test_traces_paths, batch_size=1, 
                               dict_floors=dict_floors_test,
                               experiment=experiment, shuffle=False, training=False)

dict_test_predictions = {}
for i, batch in enumerate(tqdm(test_data_generator)):
    arr_predictions, arr_max_lengths = evaluateTraces(batch, model, batch_size=1)
    trace = list_test_traces[i]    
    dict_test_predictions[trace] = {
        'site' : dict_test_site_floor_traces[trace][0],
        'floor' : dict_test_site_floor_traces[trace][1],
        'predictions' : arr_predictions
    }
    
if not os.path.exists(PATH_PREDICTIONS + load_model_name):
    os.mkdir(PATH_PREDICTIONS + load_model_name)

with open(f'{PATH_PREDICTIONS}{load_model_name}/predictions.pkl', 'wb') as f:
    pickle.dump(dict_test_predictions, f)     
    
#################################################